In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
from itertools import combinations

In [2]:
PARAMETROS = {
    'task_type' : 'CPU',
    'has_time' : True,
    'silent' : True,
    'depth': 4,
    'early_stopping_rounds': 2,
    'iterations' : 100
}

In [3]:
#APERTURA ARCHIVOS
test = pd.read_pickle("Archivos/Arboles_validacion.pkl")
prediccion_arbol = pd.read_csv("Archivos/prediccion_naive_bayes_val.csv")
prediccion_red   = pd.read_csv("Archivos/prediccion_neurona_validacion.csv")
prediccion_arbol_test = pd.read_csv("Archivos/prediccion_naive_bayes.csv")
prediccion_red_test   = pd.read_csv("Archivos/prediccion_neurona_test.csv")

In [4]:
test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Buro_Approved_by_Billing_Country_mean,Buro_Approved_by_Billing_Country_std,Opportunity_Duration_by_Product_Family_mean,Opportunity_Duration_by_Product_Family_std,Total_Amount_by_Product_Family_mean,Total_Amount_by_Product_Family_std,Buro_Approved_by_Product_Family,Family_Duration,Region_Duration,Territory_Duration
0,12173,APAC,Vietnam,1,0,1,0,0,Bureaucratic_Code_5,2018-01-02,...,0.162162,0.373684,104.057234,104.814880,1.642425e+06,6.536893e+06,0.468494,-458.0,-2072,-702.0
1,17593,EMEA,Germany,1,1,0,0,0,Bureaucratic_Code_4,2013-07-27,...,0.375380,0.484344,183.600324,139.607590,9.251233e+05,2.515626e+06,0.472919,-714.0,-1106,-1495.0
2,19868,EMEA,Germany,1,1,0,0,0,Bureaucratic_Code_1,2014-03-10,...,0.375380,0.484344,121.755396,135.953158,1.509513e+06,9.694768e+06,0.495152,-256.0,-1106,-1495.0
3,12179,Middle East,None,1,1,1,1,0,Bureaucratic_Code_4,2018-01-03,...,1.000000,0.000000,183.600324,139.607590,9.251233e+05,2.515626e+06,0.472919,-714.0,-855,-2038.0
4,12023,Americas,Central US,1,1,0,0,0,Bureaucratic_Code_4,2017-12-19,...,0.340867,0.474064,104.057234,104.814880,1.642425e+06,6.536893e+06,0.468494,-458.0,-1795,-784.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4510,22652,Japan,Japan,1,1,0,0,0,Bureaucratic_Code_4,2013-09-23,...,0.151948,0.359009,165.592593,132.988943,4.029101e+06,1.412737e+07,0.472644,-483.0,-1353,-1349.0
4511,20803,Americas,NW America,1,1,1,1,0,Bureaucratic_Code_4,2013-07-27,...,0.340867,0.474064,156.610889,129.519009,2.765150e+05,6.380753e+05,0.464882,-713.0,-1795,-1795.0
4512,5850,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,2016-06-15,...,0.340867,0.474064,38.175676,42.014791,9.527475e+06,4.935771e+07,0.488293,NaN,-1795,-1854.0
4513,5839,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,2016-06-15,...,0.340867,0.474064,46.367347,49.671292,8.221036e+03,2.192223e+04,0.505076,NaN,-1795,-1854.0


In [5]:
#ORDENO ARCHIVOS PARA QUE COINCIDA OPPORTUNITY_ID
test = test.sort_values('Opportunity_ID')
prediccion_arbol = prediccion_arbol.sort_values('Opportunity_ID')
prediccion_red   = prediccion_red.sort_values('Opportunity_ID')
test_label = test['Stage']

In [7]:
prediccion_red.count()

Opportunity_ID    4515
Target            4515
dtype: int64

In [8]:
prediccion_arbol.count()

Opportunity_ID    4515
Target            4515
dtype: int64

In [9]:
#MAX
def ensamble_max():
    preds = prediccion_arbol['Target'].combine(prediccion_red['Target'], max, 0)
    print('Logloss Max: ', skl.metrics.log_loss(test_label,preds))
    preds = prediccion_arbol_test['Target'].combine(prediccion_red_test['Target'], max, 0)
    return preds

In [10]:
#MIN
def ensamble_min():
    preds = prediccion_arbol['Target'].combine(prediccion_red['Target'], min, 0)
    print('Logloss Min: ', skl.metrics.log_loss(test_label,preds))
    preds = prediccion_arbol_test['Target'].combine(prediccion_red_test['Target'], min, 0)
    return preds

In [11]:
#COMBINACION LINEAL
def ensamble_cl():
    min_a = -1
    min_b = -1
    min_logloss = 1000

    for a in range(0,100):
        for b in range(0,100):
            if(a == 0 & b == 0): continue
            preds = (a*prediccion_arbol['Target'] + b*prediccion_red['Target'])/(a+b)
            logloss = skl.metrics.log_loss(test_label,preds)
            if(logloss < min_logloss):
                min_logloss = logloss
                min_a = a
                min_b = b
    print("Logloss CL: ", min_logloss)
    print("min_a: ", min_a)
    print("min_b: ", min_b)
    preds = (min_a*prediccion_arbol_test['Target'] + min_b*prediccion_red_test['Target'])/(min_a+min_b)
    return preds

In [12]:
#KNN
def ensamble_knn():
    temp = pd.DataFrame()
    temp['Arbol'] = prediccion_arbol['Target']
    temp['Red'] = prediccion_red['Target']
    temp['Stage'] = test_label
    entrenamiento = temp.iloc[:4000,:]
    validacion = temp.iloc[4001:,:]
    entrenamiento_label = entrenamiento.pop('Stage')
    classifier = KNeighborsClassifier(n_neighbors=300)
    classifier.fit(entrenamiento, entrenamiento_label)
    preds = classifier.predict_proba(validacion[['Arbol','Red']])
    print("Logloss KNN: ", skl.metrics.log_loss(validacion['Stage'],preds))

    classifier.fit(temp[['Arbol','Red']], temp['Stage'])
    temp = pd.DataFrame()
    temp['Arbol'] = prediccion_arbol_test['Target']
    temp['Red'] = prediccion_red_test['Target']
    preds = classifier.predict_proba(temp)
    preds = pd.Series([p[1] for p in preds])
    return preds

In [13]:
#Catboost
def ensamble_catboost():
    temp = pd.DataFrame()
    temp['Arbol'] = prediccion_arbol['Target']
    temp['Red'] = prediccion_red['Target']
    temp['Stage'] = test_label
    entrenamiento = temp.iloc[:4000,:]
    validacion = temp.iloc[4001:,:]
    entrenamiento_label = entrenamiento.pop('Stage')
    entrenamiento_pool = cb.Pool(entrenamiento, entrenamiento_label)
    test_pool = cb.Pool(validacion)
    
    model = cb.CatBoostClassifier(**PARAMETROS)
    model.fit(entrenamiento_pool)
    preds = model.predict_proba(test_pool)
    preds = pd.Series([p[1] for p in preds])
    print("Logloss Cat: ", skl.metrics.log_loss(validacion['Stage'],preds))

    model.fit(temp[['Arbol','Red']], temp['Stage'])
    temp = pd.DataFrame()
    temp['Arbol'] = prediccion_arbol_test['Target']
    temp['Red'] = prediccion_red_test['Target']
    test_pool = cb.Pool(temp)
    preds = model.predict_proba(test_pool)
    preds = pd.Series([p[1] for p in preds]) 
    return preds


In [14]:
(prediccion_red['Target'] >= np.inf).value_counts()

False    4515
Name: Target, dtype: int64

In [15]:
#CALCULO Y SELECCION DE PREDS
preds_max = ensamble_max()
preds_min = ensamble_min()
preds_cl  = ensamble_cl()
preds_knn = ensamble_knn()
preds_catboost = ensamble_catboost()

Logloss Max:  0.36552889445835457
Logloss Min:  0.39454965947733617
Logloss CL:  0.19166015977318812
min_a:  2
min_b:  53
Logloss KNN:  0.5938532437367339
Logloss Cat:  0.5952669905007728


In [16]:
#GUARDADO DE ARCHIVOS
preds = preds_catboost
resultados = pd.DataFrame()
resultados['Opportunity_ID'] = prediccion_arbol_test['Opportunity_ID']
resultados['Target'] = pd.Series(preds)  
resultados.to_csv("Archivos/prediccion_ensamble_red_naive_bayes.csv", index=False)

In [17]:
#REFINAMIENTO DE CATBOOST
#En caso de igualar/superar a KNN

from catboost import CatBoostClassifier
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold

temp = pd.DataFrame()
temp['Arbol'] = prediccion_arbol['Target']
temp['Red'] = prediccion_red['Target']
temp['Stage'] = test_label
entrenamiento = temp.iloc[:4000,:]
entrenamiento_label = entrenamiento.pop('Stage')
validacion = temp.iloc[4001:,:]
    
# Classifier
bayes_cv_tuner = BayesSearchCV(
estimator = CatBoostClassifier(
silent=True,task_type="GPU", devices='0:1', bootstrap_type='MVS'
),
search_spaces = {
        "iterations" : (2, 10),
        'learning_rate': Real(low=0.01, high=1, prior='log-uniform'),
        "random_seed" : (1,40000),
        "l2_leaf_reg" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'subsample': Real(low=0.01, high=1, prior='uniform'),
        "random_strength" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'depth': (1, 10),
        #"rsm" : Real(low=0.01, high=1, prior='uniform'),
        "early_stopping_rounds" : (1, 5),
        "border_count" : (1,65535),
        "bagging_temperature" : Real(low=1e-9,high=1000,prior='log-uniform'),
        "scale_pos_weight" : Real(low = 0.01,high = 1.0,prior="uniform")
},
cv = skl.model_selection.TimeSeriesSplit(),
n_jobs = 1,
n_iter = 6,
verbose = 1,
refit = True,
random_state = 72
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
resultCAT = bayes_cv_tuner.fit(entrenamiento, entrenamiento_label, callback=status_print)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model #1
Best ROC-AUC: 0.4222
Best params: OrderedDict([('bagging_temperature', 7.385095136875193e-08), ('border_count', 60371), ('depth', 4), ('early_stopping_rounds', 2), ('iterations', 6), ('l2_leaf_reg', 0.10952643394246851), ('learning_rate', 0.030724310795859633), ('random_seed', 30688), ('random_strength', 0.013384337685341417), ('scale_pos_weight', 0.23362736443976684), ('subsample', 0.5319710298103609)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model #2
Best ROC-AUC: 0.4366
Best params: OrderedDict([('bagging_temperature', 23.26237586333095), ('border_count', 57654), ('depth', 6), ('early_stopping_rounds', 5), ('iterations', 6), ('l2_leaf_reg', 0.04833658474449306), ('learning_rate', 0.37521472228021935), ('random_seed', 30339), ('random_strength', 0.0001401982913765261), ('scale_pos_weight', 0.19325213685877413), ('subsample', 0.7398954441876785)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Model #4
Best ROC-AUC: 0.6024
Best params: OrderedDict([('bagging_temperature', 1.1063407351624084), ('border_count', 7137), ('depth', 9), ('early_stopping_rounds', 4), ('iterations', 7), ('l2_leaf_reg', 0.17260178161990627), ('learning_rate', 0.027307511040591478), ('random_seed', 5712), ('random_strength', 2.3464965049108726e-07), ('scale_pos_weight', 0.819600044656442), ('subsample', 0.9880746674599722)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model #5
Best ROC-AUC: 0.6024
Best params: OrderedDict([('bagging_temperature', 1.1063407351624084), ('border_count', 7137), ('depth', 9), ('early_stopping_rounds', 4), ('iterations', 7), ('l2_leaf_reg', 0.17260178161990627), ('learning_rate', 0.027307511040591478), ('random_seed', 5712), ('random_strength', 2.3464965049108726e-07), ('scale_pos_weight', 0.819600044656442), ('subsample', 0.9880746674599722)])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s finished


Model #6
Best ROC-AUC: 0.6024
Best params: OrderedDict([('bagging_temperature', 1.1063407351624084), ('border_count', 7137), ('depth', 9), ('early_stopping_rounds', 4), ('iterations', 7), ('l2_leaf_reg', 0.17260178161990627), ('learning_rate', 0.027307511040591478), ('random_seed', 5712), ('random_strength', 2.3464965049108726e-07), ('scale_pos_weight', 0.819600044656442), ('subsample', 0.9880746674599722)])

